# 🔍 AI PK System - Google Colab Version

AI A ↔ AI B cross-verification with OpenAI arbitration

## Setup Instructions:
1. Add your API keys in **Colab Secrets** (🔑 icon in left sidebar)
2. Add your ngrok token as `NGROK_TOKEN`
3. Run all cells in order

In [ ]:
# Cell 1: Install dependencies
!pip install -q flask pyngrok google-generativeai openai

In [ ]:
# Cell 2: Load API keys from Colab Secrets
from google.colab import userdata
from pyngrok import ngrok

# Configure ngrok
ngrok.set_auth_token(userdata.get('NGROK_TOKEN'))

# Load API keys
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
GROK_API_KEY = userdata.get('GROK_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
DOUBAO_API_KEY = userdata.get('DOUBAO_API_KEY')
DOUBAO_ENDPOINT_ID = userdata.get('DOUBAO_ENDPOINT_ID')
QWEN_API_KEY = userdata.get('QWEN_API_KEY')

print('✅ API keys loaded from Colab Secrets')

In [ ]:
# Cell 3: Flask App Setup
import json
from flask import Flask, request, jsonify, Response
import google.generativeai as genai
from openai import OpenAI

app = Flask(__name__)

# Model configuration
MODEL_GEMINI = "gemini-2.5-flash"
MODEL_GROK = "grok-4-1-fast-reasoning"
MODEL_OPENAI = "gpt-4o"
MODEL_DOUBAO = "doubao-pro-32k"
MODEL_QWEN = "qwen-max"
MAX_REVISION_ROUNDS = 3

AVAILABLE_MODELS = {
    'gemini': {'name': 'Gemini', 'display_name': 'Gemini (Google)', 'icon': 'G', 'color': '#4285f4'},
    'grok': {'name': 'Grok', 'display_name': 'Grok (xAI)', 'icon': 'X', 'color': '#ff6b35'},
    'doubao': {'name': 'Doubao', 'display_name': '豆包 (ByteDance)', 'icon': '豆', 'color': '#00d4aa'},
    'qwen': {'name': 'Qwen', 'display_name': '千问 (Alibaba)', 'icon': '千', 'color': '#ff5722'}
}

# Configure clients
genai.configure(api_key=GEMINI_API_KEY)
gemini_model = genai.GenerativeModel(MODEL_GEMINI)
grok_client = OpenAI(api_key=GROK_API_KEY, base_url="https://api.x.ai/v1")
openai_client = OpenAI(api_key=OPENAI_API_KEY)
doubao_client = OpenAI(api_key=DOUBAO_API_KEY, base_url="https://ark.cn-beijing.volces.com/api/v3")
qwen_client = OpenAI(api_key=QWEN_API_KEY, base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

print('✅ AI clients configured')

In [ ]:
# Cell 4: AI Model Functions
def ask_gemini(prompt):
    try:
        response = gemini_model.generate_content(prompt)
        return True, response.text
    except Exception as e:
        return False, f"Gemini error: {e}"

def ask_grok(user_message):
    try:
        response = grok_client.chat.completions.create(
            model=MODEL_GROK, messages=[{"role": "user", "content": user_message}])
        return True, response.choices[0].message.content
    except Exception as e:
        return False, f"Grok error: {e}"

def ask_doubao(user_message):
    try:
        response = doubao_client.chat.completions.create(
            model=DOUBAO_ENDPOINT_ID, messages=[{"role": "user", "content": user_message}])
        return True, response.choices[0].message.content
    except Exception as e:
        return False, f"Doubao error: {e}"

def ask_qwen(user_message):
    try:
        response = qwen_client.chat.completions.create(
            model=MODEL_QWEN, messages=[{"role": "user", "content": user_message}])
        return True, response.choices[0].message.content
    except Exception as e:
        return False, f"Qwen error: {e}"

def ask_openai(prompt):
    try:
        response = openai_client.chat.completions.create(
            model=MODEL_OPENAI,
            messages=[
                {"role": "system", "content": "你是一个严格的事实核查仲裁者。你的判断必须准确，尤其要识别AI编造虚假信息（幻觉）的情况。"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1)
        return True, response.choices[0].message.content
    except Exception as e:
        return False, f"OpenAI error: {e}"

def ask_model(model_id, prompt):
    if model_id == 'gemini': return ask_gemini(prompt)
    elif model_id == 'grok': return ask_grok(prompt)
    elif model_id == 'doubao': return ask_doubao(prompt)
    elif model_id == 'qwen': return ask_qwen(prompt)
    else: return False, f"Unknown model: {model_id}"

def get_model_name(model_id):
    return AVAILABLE_MODELS.get(model_id, {}).get('name', model_id)

print('✅ AI functions defined')

In [ ]:
# Cell 5: Prompt Templates
def create_verification_prompt(question, answer):
    return f"""
    我有一个问题: "{question}"
    另一个 AI 模型给出了以下回答:
    ---
    {answer}
    ---
    请作为一名严厉、客观的事实核查员，评价上述回答。
    你需要：
    1. 检查是否有明显的事实性错误。
    2. 指出逻辑漏洞。
    3. 如果回答非常完美，请明确说"完美"或"无需修改"。
    重要说明：不同AI有不同的知识截止日期。如果你无法核实某些近期事件，这并不意味着对方编造了信息。
    请直接给出你的评审意见。
    """

def create_arbiter_prompt(question, answer, review):
    return f"""你是一个公正严格的仲裁者。分析以下评审意见，判断回答是否需要修正。
原始问题: "{question}"
AI 的回答:
---
{answer}
---
另一个 AI 的评审意见:
---
{review}
---
请严格按照以下JSON格式回复：
{{"needs_revision": true或false, "reason": "简短说明理由", "corrections": "如需修订，列出具体修正要点；否则为空"}}
"""

def create_revision_prompt(question, original_answer, corrections):
    return f"""
针对问题: "{question}"
你之前的回答存在问题:
---
{original_answer}
---
评审发现以下问题需要修正:
---
{corrections}
---
请给出修正后的回答。
"""

import re
def parse_arbiter_response(response):
    try:
        result = json.loads(response)
        return result.get("needs_revision", False), result.get("corrections", "") or result.get("reason", "")
    except json.JSONDecodeError:
        pass
    json_match = re.search(r'\{[^{}]*"needs_revision"[^{}]*\}', response, re.DOTALL)
    if json_match:
        try:
            result = json.loads(json_match.group())
            return result.get("needs_revision", False), result.get("corrections", "") or result.get("reason", "")
        except: pass
    if "完美" in response or "无需修改" in response:
        return False, ""
    return False, ""

print('✅ Prompt templates defined')

In [ ]:
# Cell 6: HTML Template (stored as string)
HTML_TEMPLATE = open('index_template.html').read() if False else '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI PK System</title>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap" rel="stylesheet">
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
    <style>
        :root { --bg-primary: #0f0f1a; --bg-secondary: #1a1a2e; --bg-card: #16162a; --text-primary: #ffffff; --text-secondary: #a0a0c0; --gemini-color: #4285f4; --grok-color: #ff6b35; --openai-color: #10a37f; --border-color: rgba(255,255,255,0.08); }
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body { font-family: 'Inter', sans-serif; background: var(--bg-primary); color: var(--text-primary); min-height: 100vh; line-height: 1.6; }
        .container { max-width: 1400px; margin: 0 auto; padding: 40px 20px; }
        .header { text-align: center; margin-bottom: 40px; }
        .header h1 { font-size: 2.5rem; font-weight: 700; background: linear-gradient(135deg, #4285f4, #ff6b35, #10a37f); -webkit-background-clip: text; -webkit-text-fill-color: transparent; margin-bottom: 10px; }
        .header p { color: var(--text-secondary); font-size: 1.1rem; }
        .input-section { background: var(--bg-card); border-radius: 20px; padding: 30px; margin-bottom: 40px; border: 1px solid var(--border-color); }
        .model-selectors { display: flex; gap: 20px; margin-bottom: 20px; }
        .model-selector { flex: 1; display: flex; flex-direction: column; gap: 8px; }
        .model-selector label { font-size: 0.85rem; color: var(--text-secondary); }
        .model-selector select { background: var(--bg-secondary); border: 1px solid var(--border-color); border-radius: 10px; padding: 12px 16px; color: var(--text-primary); font-size: 0.95rem; cursor: pointer; }
        .input-wrapper { display: flex; gap: 15px; }
        .question-input { flex: 1; background: var(--bg-secondary); border: 1px solid var(--border-color); border-radius: 12px; padding: 18px 24px; color: var(--text-primary); font-size: 1rem; outline: none; }
        .submit-btn { background: linear-gradient(135deg, #4285f4, #34a853); border: none; border-radius: 12px; padding: 18px 40px; color: white; font-size: 1rem; font-weight: 600; cursor: pointer; }
        .submit-btn:disabled { opacity: 0.6; cursor: not-allowed; }
        .status-bar { background: var(--bg-secondary); border-radius: 12px; padding: 15px 25px; margin-bottom: 30px; display: flex; align-items: center; gap: 15px; opacity: 0; }
        .status-bar.active { opacity: 1; }
        .spinner { width: 20px; height: 20px; border: 2px solid var(--border-color); border-top-color: var(--gemini-color); border-radius: 50%; animation: spin 1s linear infinite; }
        @keyframes spin { to { transform: rotate(360deg); } }
        .response-grid { display: grid; grid-template-columns: 1fr 1fr; gap: 30px; margin-bottom: 30px; }
        .response-card { background: var(--bg-card); border-radius: 20px; overflow: hidden; border: 1px solid var(--border-color); }
        .response-header { padding: 20px 25px; display: flex; align-items: center; gap: 12px; }
        .ai-icon { width: 40px; height: 40px; background: rgba(255,255,255,0.2); border-radius: 10px; display: flex; align-items: center; justify-content: center; font-weight: 700; font-size: 1.2rem; }
        .response-title { font-size: 1.3rem; font-weight: 600; }
        .response-content { padding: 25px; min-height: 200px; max-height: 500px; overflow-y: auto; }
        .arbiter-section { background: var(--bg-card); border-radius: 20px; overflow: hidden; border: 1px solid var(--border-color); }
        .arbiter-header { padding: 20px 25px; background: linear-gradient(135deg, #10a37f, #1a7f64); display: flex; align-items: center; gap: 12px; }
        .arbiter-content { padding: 25px; }
        .arbiter-grid { display: grid; grid-template-columns: 1fr 1fr; gap: 20px; }
        .arbiter-item { background: var(--bg-secondary); border-radius: 12px; padding: 20px; }
        .consensus-banner { text-align: center; padding: 30px; margin-top: 20px; }
        .consensus-badge { display: inline-flex; align-items: center; gap: 10px; padding: 15px 30px; border-radius: 50px; font-weight: 600; }
        .consensus-badge.success { background: rgba(16,163,127,0.2); color: #10a37f; }
        .consensus-badge.warning { background: rgba(255,193,7,0.2); color: #ffc107; }
        .markdown-content h1,.markdown-content h2,.markdown-content h3 { margin: 1em 0 0.5em; }
        .markdown-content p { margin-bottom: 1em; }
        .markdown-content code { background: rgba(255,255,255,0.1); padding: 0.2em 0.4em; border-radius: 4px; }
        .markdown-content pre { background: var(--bg-secondary); padding: 1em; border-radius: 8px; overflow-x: auto; }
        .status-badge { padding: 6px 14px; border-radius: 20px; font-size: 0.85rem; font-weight: 600; }
        .status-badge.no-revision { background: rgba(16,163,127,0.2); color: #10a37f; }
        .status-badge.needs-revision { background: rgba(255,107,53,0.2); color: #ff6b35; }
        .revision-badge { display: inline-block; background: rgba(255,193,7,0.2); color: #ffc107; padding: 4px 12px; border-radius: 20px; font-size: 0.75rem; margin-left: auto; }
        @media (max-width: 900px) { .response-grid, .arbiter-grid { grid-template-columns: 1fr; } .input-wrapper { flex-direction: column; } }
    </style>
</head>
<body>
    <div class="container">
        <header class="header">
            <h1>🔍 AI PK System</h1>
            <p>AI A ↔ AI B cross-verification with OpenAI arbitration</p>
        </header>
        <div class="input-section">
            <div class="model-selectors">
                <div class="model-selector"><label>Model A</label><select id="modelA"><option value="gemini">Gemini</option><option value="grok">Grok</option><option value="doubao">豆包</option><option value="qwen">千问</option></select></div>
                <div class="model-selector"><label>Model B</label><select id="modelB"><option value="gemini">Gemini</option><option value="grok" selected>Grok</option><option value="doubao">豆包</option><option value="qwen">千问</option></select></div>
            </div>
            <div class="input-wrapper">
                <input type="text" class="question-input" id="questionInput" placeholder="Enter your question...">
                <button class="submit-btn" id="submitBtn" onclick="submitQuestion()">Verify</button>
            </div>
        </div>
        <div class="status-bar" id="statusBar"><div class="spinner"></div><span id="statusText">Processing...</span></div>
        <div class="response-grid">
            <div class="response-card"><div class="response-header" id="headerA" style="background:linear-gradient(135deg,#4285f4,#34a853)"><div class="ai-icon" id="iconA">G</div><span class="response-title" id="titleA">Model A</span><span class="revision-badge" id="badgeA" style="display:none"></span></div><div class="response-content"><div id="responseA"><span style="color:var(--text-secondary);font-style:italic">Model A response...</span></div></div></div>
            <div class="response-card"><div class="response-header" id="headerB" style="background:linear-gradient(135deg,#ff6b35,#ff8c42)"><div class="ai-icon" id="iconB">X</div><span class="response-title" id="titleB">Model B</span><span class="revision-badge" id="badgeB" style="display:none"></span></div><div class="response-content"><div id="responseB"><span style="color:var(--text-secondary);font-style:italic">Model B response...</span></div></div></div>
        </div>
        <div class="arbiter-section">
            <div class="arbiter-header"><div class="ai-icon">⚖️</div><span class="response-title">OpenAI Arbiter</span></div>
            <div class="arbiter-content">
                <div class="arbiter-grid">
                    <div class="arbiter-item"><div style="margin-bottom:10px;color:var(--text-secondary)">Analysis of Model A</div><div id="arbiterA"><span style="color:var(--text-secondary);font-style:italic">Waiting...</span></div></div>
                    <div class="arbiter-item"><div style="margin-bottom:10px;color:var(--text-secondary)">Analysis of Model B</div><div id="arbiterB"><span style="color:var(--text-secondary);font-style:italic">Waiting...</span></div></div>
                </div>
                <div class="consensus-banner" id="consensusBanner" style="display:none"><div class="consensus-badge" id="consensusBadge"></div></div>
            </div>
        </div>
    </div>
    <script>
        const modelColors = { gemini: {gradient:'linear-gradient(135deg,#4285f4,#34a853)',icon:'G'}, grok: {gradient:'linear-gradient(135deg,#ff6b35,#ff8c42)',icon:'X'}, doubao: {gradient:'linear-gradient(135deg,#00d4aa,#00a085)',icon:'豆'}, qwen: {gradient:'linear-gradient(135deg,#ff5722,#e64a19)',icon:'千'} };
        let revisionsA=0, revisionsB=0;
        function setStatus(msg,show=true) { document.getElementById('statusBar').classList.toggle('active',show); document.getElementById('statusText').textContent=msg; }
        function renderMarkdown(t) { try{return marked.parse(t||'')} catch(e){return t} }
        function formatArbiter(raw) {
            try { let p=JSON.parse(raw); return `<span class="status-badge ${p.needs_revision?'needs-revision':'no-revision'}">${p.needs_revision?'⚠️ Needs Revision':'✓ OK'}</span><p style="margin-top:10px">${p.reason||''}</p>`; } catch(e) { return renderMarkdown(raw); }
        }
        async function submitQuestion() {
            const q=document.getElementById('questionInput').value.trim();
            if(!q) return alert('Enter a question');
            document.getElementById('submitBtn').disabled=true; revisionsA=0; revisionsB=0;
            document.getElementById('badgeA').style.display='none'; document.getElementById('badgeB').style.display='none';
            setStatus('Initializing...');
            try {
                const res=await fetch('/verify',{method:'POST',headers:{'Content-Type':'application/json'},body:JSON.stringify({question:q,model_a:document.getElementById('modelA').value,model_b:document.getElementById('modelB').value})});
                const reader=res.body.getReader(), decoder=new TextDecoder();
                while(true) { const{done,value}=await reader.read(); if(done)break; decoder.decode(value).split('\\n').filter(l=>l.trim()).forEach(line=>{ try{handleUpdate(JSON.parse(line))}catch(e){} }); }
            } catch(e) { setStatus('Error: '+e.message); }
            document.getElementById('submitBtn').disabled=false; setStatus('',false);
        }
        function handleUpdate(d) {
            if(d.type=='models') { ['A','B'].forEach((x,i)=>{ let m=i?d.model_b:d.model_a,c=modelColors[m]; document.getElementById('header'+x).style.background=c.gradient; document.getElementById('icon'+x).textContent=c.icon; document.getElementById('title'+x).textContent=m; }); }
            else if(d.type=='status') setStatus(d.message);
            else if(d.type=='model_a_initial') { let e=document.getElementById('responseA'); e.innerHTML=renderMarkdown(d.content); e.classList.add('markdown-content'); }
            else if(d.type=='model_b_initial') { let e=document.getElementById('responseB'); e.innerHTML=renderMarkdown(d.content); e.classList.add('markdown-content'); }
            else if(d.type=='model_a_revised') { revisionsA++; document.getElementById('responseA').innerHTML=renderMarkdown(d.content); document.getElementById('badgeA').textContent='Revised x'+revisionsA; document.getElementById('badgeA').style.display='inline-block'; }
            else if(d.type=='model_b_revised') { revisionsB++; document.getElementById('responseB').innerHTML=renderMarkdown(d.content); document.getElementById('badgeB').textContent='Revised x'+revisionsB; document.getElementById('badgeB').style.display='inline-block'; }
            else if(d.type=='arbiter') { document.getElementById('arbiterA').innerHTML=formatArbiter(d.model_a_analysis); document.getElementById('arbiterB').innerHTML=formatArbiter(d.model_b_analysis); }
            else if(d.type=='consensus') { let b=document.getElementById('consensusBanner'),bg=document.getElementById('consensusBadge'); b.style.display='block'; bg.className='consensus-badge '+(d.reached?'success':'warning'); bg.innerHTML=(d.reached?'✓ Consensus Round '+d.round:'⚠ Max Rounds Reached'); }
        }
        document.getElementById('questionInput').addEventListener('keypress',e=>{if(e.key=='Enter')submitQuestion();});
    </script>
</body>
</html>'''

print(f'✅ HTML template loaded ({len(HTML_TEMPLATE)} bytes)')

In [ ]:
# Cell 7: Flask Routes
@app.route('/')
def index():
    return HTML_TEMPLATE

@app.route('/models')
def get_models():
    return jsonify(AVAILABLE_MODELS)

@app.route('/verify', methods=['POST'])
def verify():
    question = request.json.get('question', '')
    model_a = request.json.get('model_a', 'gemini')
    model_b = request.json.get('model_b', 'grok')
    if not question.strip():
        return jsonify({'error': 'Please enter a question'}), 400
    if model_a not in AVAILABLE_MODELS: model_a = 'gemini'
    if model_b not in AVAILABLE_MODELS: model_b = 'grok'

    def generate():
        yield json.dumps({'type':'models','model_a':model_a,'model_b':model_b,'model_a_info':AVAILABLE_MODELS[model_a],'model_b_info':AVAILABLE_MODELS[model_b]})+'\n'
        yield json.dumps({'type':'status','message':f'Getting {get_model_name(model_a)} response...'})+'\n'
        ok_a, answer_a = ask_model(model_a, question)
        if not ok_a: yield json.dumps({'type':'error','source':'model_a','message':answer_a})+'\n'; return
        yield json.dumps({'type':'model_a_initial','content':answer_a})+'\n'
        yield json.dumps({'type':'status','message':f'Getting {get_model_name(model_b)} response...'})+'\n'
        ok_b, answer_b = ask_model(model_b, question)
        if not ok_b: yield json.dumps({'type':'error','source':'model_b','message':answer_b})+'\n'; return
        yield json.dumps({'type':'model_b_initial','content':answer_b})+'\n'
        round_num, consensus = 1, False
        while round_num <= MAX_REVISION_ROUNDS and not consensus:
            yield json.dumps({'type':'status','message':f'Round {round_num}: Cross-verification...'})+'\n'
            ok_rb, review_b = ask_model(model_b, create_verification_prompt(question, answer_a))
            if not ok_rb: yield json.dumps({'type':'error','message':review_b})+'\n'; return
            ok_ra, review_a = ask_model(model_a, create_verification_prompt(question, answer_b))
            if not ok_ra: yield json.dumps({'type':'error','message':review_a})+'\n'; return
            yield json.dumps({'type':'status','message':f'Round {round_num}: OpenAI arbiter analyzing...'})+'\n'
            ok_arb_a, arbiter_a = ask_openai(create_arbiter_prompt(question, answer_a, review_b))
            ok_arb_b, arbiter_b = ask_openai(create_arbiter_prompt(question, answer_b, review_a))
            a_needs, a_corr = parse_arbiter_response(arbiter_a)
            b_needs, b_corr = parse_arbiter_response(arbiter_b)
            yield json.dumps({'type':'arbiter','round':round_num,'model_a_analysis':arbiter_a,'model_b_analysis':arbiter_b})+'\n'
            if not a_needs and not b_needs:
                consensus = True
                yield json.dumps({'type':'consensus','reached':True,'round':round_num})+'\n'
            else:
                if a_needs:
                    ok_rev_a, answer_a = ask_model(model_a, create_revision_prompt(question, answer_a, a_corr))
                    yield json.dumps({'type':'model_a_revised','round':round_num,'content':answer_a})+'\n'
                if b_needs:
                    ok_rev_b, answer_b = ask_model(model_b, create_revision_prompt(question, answer_b, b_corr))
                    yield json.dumps({'type':'model_b_revised','round':round_num,'content':answer_b})+'\n'
                round_num += 1
        if not consensus:
            yield json.dumps({'type':'consensus','reached':False,'round':round_num-1})+'\n'
        yield json.dumps({'type':'final','model_a':answer_a,'model_b':answer_b,'consensus':consensus})+'\n'
    return Response(generate(), mimetype='application/x-ndjson')

print('✅ Flask routes defined')

In [ ]:
# Cell 8: Start the server with ngrok
import threading

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print(f'\n🚀 Public URL: {public_url}')
print('\n⚠️ Keep this notebook running! Click the URL above to access the AI PK System.')

# Run Flask in a thread
threading.Thread(target=lambda: app.run(port=5000, use_reloader=False)).start()